In [1]:
import math
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, q_dim, k_dim, v_dim, hidden_dim, num_head, dropout):
        super(MultiHeadAttention, self).__init__()
        self.num_head = num_head
        self.hidden_dim = hidden_dim
        self.W_q = nn.Linear(q_dim, hidden_dim)
        self.W_k = nn.Linear(k_dim, hidden_dim)
        self.W_v = nn.Linear(v_dim, hidden_dim)
        self.W_o = nn.Linear(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self, query, key, value, valid_len = None, causal = True):
        head_dim = self.hidden_dim // self.num_head
        B, Lq, _ = query.shape
        _, Lk, _ = key.shape
        _, Lv, _ = value.shape
        Q = self.W_q(query)
        K = self.W_k(key)
        V = self.W_v(value)
        Q = Q.reshape(B, Lq, self.num_head, head_dim).permute(0,2,1,3)
        K = K.reshape(B, Lk, self.num_head, head_dim).permute(0,2,1,3)
        V = V.reshape(B, Lv, self.num_head, head_dim).permute(0,2,1,3)
        scale = Q @ K.transpose(2,3) / (head_dim**0.5)
        if valid_len != None:
            mask = torch.arange(Lk)[None, None, None, : ] >= valid_len[:, None, None, None]
            scale = scale.masked_fill(mask, -1e6)
        if causal:
            causal_mask = torch.zeros(Lq,Lk).triu(1).bool()
            scale = scale.masked_fill(causal_mask[None, None, :,:], -1e6)
        weight = F.softmax(scale, dim = -1) 
        weight = self.dropout(weight)
        out = weight @ V
        O = out.reshape(B, Lq, self.hidden_dim)
        return self.W_o(O)

In [3]:
class FeedForward(nn.Module):
    def __init__(self, hidden_dim, ff_dim, dropout):
        super().__init()
        self.fc1 = nn.Linear(hidden_dim, ff_dim)
        self.fc2 = nn.Linear(ff_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        return  self.fc2(self.dropout(F.relu(self.fc1(x))))

In [4]:
class GPTBlock(nn.Module):
    def __init__(self, hidden_dim, num_head, ff_dim, dropout):
        super().__init()
        self.attention = MultiHeadAttention(hidden_dim, hidden_dim, hidden_dim, hidden_dim, num_head, dropout)
        self.mlp = FeedForward(hidden_dim, ff_dim, dropout)
        self.norm1 = nn.LayerNorm(hidden_dim)
        self.norm2 = nn.LayerNorm(hidden_dim)
    def forward(self, x):
        att = self.attention(x, x, x, causal = True)
        x = self.norm1(x + att)
        m = self.mlp(x)
        x = self.norm2(x + m)
        return x    

In [5]:
class GPT(nn.Module):
    def __init__(self, vocab_size, hidden_dim, num_layer, num_head, ff_dim, max_len=1000, dropout = 0.5):
        super().__init__()
        self.num_layer = num_layer
        self.tokenEmbedding = nn.Embedding(vocab_size, hidden_dim)
        self.positionEmbedding = nn.Embedding(max_len, hidden_dim)
        self.decoder = nn.ModuleList([GPTBlock(hidden_dim, num_head, ff_dim, dropout) for _ in range(num_layer)])
        self.out = nn.Linear(hidden_dim, vocab_size)
    def forward(self, tokens):
        B, L = tokens.shape
        pos = torch.arange(L).unsqueeze(0).expand(B, L)
        X = self.tokenEmbedding(tokens) + self.positionEmbedding(pos)
        for layer in self.decoder:
            X = layer(X)
        X = self.out(X)
        return X